# Playground for sparse grids

In [1]:
import numpy as np
import pandas as pd
import respy as rp
from respy.shared import calculate_expected_value_functions

## Load the example model

In [2]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
optim_paras, _ = rp.pre_processing.model_processing.process_params_and_options(params, options)

In [3]:
solve = rp.get_solve_func(params, options)
state_space = solve(params)

## State Space

The state space is captured in this pandas DataFrame. It covers all states over all periods with experiences of three alternatives and the choice in the previous period.

In [4]:
states = pd.DataFrame(state_space.core, columns=["period", "exp_a", "exp_b", "exp_edu", "lagged_choice_1"])
states.describe()

,period,exp_a,exp_b,exp_edu,lagged_choice_1
count,317367.000000,317367.000000,317367.000000,317367.000000,317367.000000
mean,30.518838,8.734456,8.734456,14.315471,1.492925
std,7.011614,7.023967,7.023967,3.029777,1.124330
min,0.000000,0.000000,0.000000,10.000000,0.000000
25%,26.000000,3.000000,3.000000,12.000000,0.000000
50%,32.000000,7.000000,7.000000,14.000000,1.000000
75%,36.000000,13.000000,13.000000,17.000000,3.000000
max,39.000000,39.000000,39.000000,20.000000,3.000000


## State Space attributes

The state space is a class with some attributes. Below we document the descriptive statistics of the rewards, wages and non-pecuniary rewards.

In [5]:
wages = pd.DataFrame(state_space.wages, columns=["wage_a", "wage_b", "wage_edu", "wage_home"])
wages.describe()

,wage_a,wage_b,wage_edu,wage_home
count,317367.000000,317367.000000,317367.0,317367.0
mean,21935.480043,25077.840710,1.0,1.0
std,4018.906860,8726.591695,0.0,0.0
min,14617.869534,9701.152773,1.0,1.0
25%,18882.672848,18324.605526,1.0,1.0
50%,21461.158524,23659.026706,1.0,1.0
75%,24636.885266,30454.833617,1.0,1.0
max,36552.322272,60475.886843,1.0,1.0


In [6]:
nonpecs = pd.DataFrame(state_space.nonpecs, columns=["nonpec_a", "nonpec_b", "nonpec_edu", "nonpec_home"])
nonpecs.describe()

,nonpec_a,nonpec_b,nonpec_edu,nonpec_home
count,317367.0,317367.0,317367.000000,317367.0
mean,0.0,0.0,-49537.551163,17750.0
std,0.0,0.0,187153.152904,0.0
min,0.0,0.0,-804000.000000,17750.0
25%,0.0,0.0,-4000.000000,17750.0
50%,0.0,0.0,-4000.000000,17750.0
75%,0.0,0.0,-4000.000000,17750.0
max,0.0,0.0,0.000000,17750.0


## The expected value function

For the simulation of the expected value function for one state, we need the wages for all alternatives, the nonpecuniary rewards, and the continuation values. Furthermore, we have a matrix of 500 x 4 shocks to simulate the expected value function via Monte Carlo simulation.

*It might be unintuitive that the wages for choices without wages, education and home production, are set to one. This reason is that shocks for choices with wages are multiplied to the wage. For choices without wage the shock is added to the nonpecuniary reward. By setting the wage to one for the latter choices we can always multiply the shock with the wage. ([see the utility function](https://github.com/OpenSourceEconomics/respy/blob/4df4dd0fd72580aa8206b2a9c91457e5aebfe5e7/respy/shared.py#L16-L51))*

In [8]:
def get_characteristics_of_single_state(idx, state_space):
    wage = state_space.wages[idx]
    nonpec = state_space.nonpecs[idx]
    
    period = state_space.core.loc[idx, "period"]
    cont_idx = idx - state_space.core.eval("period < @period").sum()

    continuation_values = state_space.get_continuation_values(indices=idx)
    
    periodic_draws = np.dot(state_space.base_draws_sol, optim_paras["shocks_cholesky"])
    periodic_draws[..., :2] = np.exp(periodic_draws[..., :2])
    
    draws = periodic_draws[period]
    
    return wage, nonpec, continuation_values, draws

In [9]:
wage, nonpec, continuation_values, draws = get_characteristics_of_single_state(1, state_space)

This function performs the Monte Carlo simulation.

In [11]:
calculate_expected_value_functions(wage, nonpec, continuation_values, draws, optim_paras["delta"])

357602.0239677464